In [ ]:
import os
import pickle
import joblib
import SimpleITK as sitk
import pandas as pd
import numpy as np
import time
import tqdm

### utility functions

In [ ]:
def generate_map(inputcsvpath, modelpath, path_to_ref_stkimg, output_dir):
    
    # get predicted csv file
    casename = os.path.basename(inputcsvpath)
    output_csvpath = os.path.join(output_dir, casename + '_predictedVals.csv')
    
    if not os.path.exists(output_csvpath):
        preds_df = generate_predictions(inputcsvpath, modelpath, output_dir)
        preds_df.to_csv(output_csvpath, index=False) # save to file
    else:
        print(output_csvpath, 'exists')
        preds_df = pd.read_csv(output_csvpath)
    
    # save predicted image
    output_mappath = os.path.join(output_dir, casename + '_predictedMap.nii.gz')
    if not os.path.exists(output_mappath):
        pred_stk = get_stkimg_from_csvfile(preds_df, path_to_ref_stkimg)
        sitk.WriteImage(pred_stk, output_mappath)
    else:
        print(output_mappath, 'exists')

def get_stkimg_from_df(df, path_to_reference_img):
    
    #load reference image
    print('-- loading referece img ')
    ref_stkimg = sitk.ReadImage(path_to_reference_img)
    refarr = sitk.GetArrayFromImage(ref_stkimg).T
    imshape = refarr.shape
    print('ref img', refarr.shape)

    print('--create prediction image..')
    if df.shape[0] == imshape[0] * imshape[1] * imshape[2]:

        vals = df['prediction'].values
        im = np.array(vals + 1.0).astype('I').reshape(refarr.shape)

    else:
        im = np.zeros(refarr.shape)
        indices = list(df.index.values)
        time.sleep(2.01)
        for i in tqdm.tqdm(range(len(indices))):
            x, y, z = [int(a) for a in df.loc[indices[i], ['x', 'y', 'z']].values]
            val = float(df.loc[indices[i], 'prediction']) +1
            im[x, y, z] = val

    stkimg = sitk.GetImageFromArray(im.T)
    stkimg.CopyInformation(ref_stkimg)

    return stkimg

def generate_predictions(df, model_features, model):
    
    preds = model.predict(df[model_features].values)
    print(preds)
    
    # report in df format
    pred_df = pd.DataFrame(index=df.index.values)
    pred_df['prediction'] = preds[:,1]
    for col in ['x', 'y', 'z']:
        pred_df[col] = df[col].values
    
    return pred_df

In [ ]:
# paths 
input_dir = os.getcwd() 
testcase_dir = os.path.join(input_dir, 'testcases')
model_dir = os.path.join(input_dir, foldername, sex, 'trained_models')
output_dir = testcase_dir

In [ ]:
# load model object 
modelname = 'rfr_gl_Mes-gl_PN' 
modelpath = os.path.join(model_dir, model_name + '.sav')
model = pickle.load(open(modelname + '.sav', 'rb'))
print(modelname, 'loaded')

# load names of features that the model was trained on
model_features = ['Enhancing','RCBV.Raw_Mean', 'RCBV.Raw_Std', 'FA.Raw_Std', 
                  'FA.Raw_Mean', 'MD.Raw_Mean', 'MD.Raw_Std', 'EPI.Raw_Mean', 
                  'EPI.Raw_Std', 'CenterFecsT2', 'MeanFecsT2', 'StdFecsT2']

print ('%s takes %d features' % (model_name, len(model_features)))


In [ ]:
# load csv of test case
testcase_name = 'MCH2868'
testcase_csvpath = os.path.join(testcase_dir, 'MCH2868_features.csv')
df = pd.read_csv(testcase_csvpath, low_memory=False)

In [ ]:
# generate predictions
pred_df = generate_predictions(df, model_features, model)

# write to csv file
output_csvpath = = os.path.join(testcase_dir, 'MCH2868_pred.csv')
pred_df.to_csv(output_csvpath, index=False)

# get predicted image
pred_stk = get_stkimg_from_df(pred_df, 'T1Gd.nii.gz')

# save image to file
output_mappath = = os.path.join(testcase_dir, testcase_name + '_predictedMapp.nii.gz')
sitk.WriteImage(pred_stk, output_mappath)